# Value by Risk at Historical Data

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data1.csv", sep = ";")

In [3]:
df.tail(5)

,date,time,Actuvia,Prost,Zhuw,Actim
34,01.11.2018,-4,58.852372,40.664455,25.566430,120.782840
35,01.12.2018,-3,59.864980,41.834031,32.063500,112.840962
36,01.01.2019,-2,63.101786,45.834003,31.549725,126.379636
37,01.02.2019,-1,61.561212,45.649542,31.533993,125.090383
38,01.03.2019,0,57.214747,43.184031,31.886190,106.582874


In [4]:
df.head(5)

,date,time,Actuvia,Prost,Zhuw,Actim
0,01.01.2016,-38,57.577623,38.981322,31.152355,113.668082
1,01.02.2016,-37,61.558370,37.585917,29.893259,112.758183
2,01.03.2016,-36,57.934659,45.124564,31.238058,123.477716
3,01.04.2016,-35,52.270286,40.699365,29.851398,112.898816
4,01.05.2016,-34,63.207726,38.415999,32.753887,111.826867


In [5]:
matrix = df[['Actuvia', 'Prost', 'Zhuw', 'Actim']]

In [6]:
matrix.tail(5)

,Actuvia,Prost,Zhuw,Actim
34,58.852372,40.664455,25.566430,120.782840
35,59.864980,41.834031,32.063500,112.840962
36,63.101786,45.834003,31.549725,126.379636
37,61.561212,45.649542,31.533993,125.090383
38,57.214747,43.184031,31.886190,106.582874


In [7]:
w = [4, 2, 6, 0.5]

In [8]:
p0 = (matrix.iloc[-1] * w).sum()

In [9]:
print("p0 : {0}".format(p0))

p0 : 559.83562419


In [10]:
P = matrix.dot(w)

In [28]:
P_date = pd.concat([P, df['date']], axis = 1)
P_date = P_date.rename({0: "P1"}, axis = 1)

In [29]:
P_date

,P1,date
0,552.021305,01.01.2016
1,557.143959,01.02.2016
2,571.154972,01.03.2016
3,526.037671,01.04.2016
4,582.099656,01.05.2016
5,579.056324,01.06.2016
6,551.372124,01.07.2016
7,555.988816,01.08.2016
8,581.450879,01.09.2016
9,567.960047,01.10.2016


In [30]:
P_date.to_csv("P1.csv")

In [11]:
Sigma = matrix.cov()

In [12]:
Sigma.head(10)

,Actuvia,Prost,Zhuw,Actim
Actuvia,11.124520,0.124383,-1.704154,-6.034819
Prost,0.124383,28.143749,2.089170,6.231386
Zhuw,-1.704154,2.089170,3.553728,-0.254739
Actim,-6.034819,6.231386,-0.254739,89.508086


In [94]:
sigma = Sigma.dot(w)

In [95]:
sigma = (sigma * w).sum()

In [96]:
sigma = math.sqrt(sigma)

In [97]:
print("sigma: {0}".format(sigma))

sigma: 19.950047728272033


In [98]:
quant_P = p0 - 1.282 * sigma

In [99]:
print("quantile of P: {0}".format(quant_P))

quantile of P: 534.2596630023552


In [100]:
quant_L = p0 - quant_P

In [101]:
print("quantile of L: {0}".format(quant_L))

quantile of L: 25.575961187644793


# Validation

In [102]:
def VAR(df):
    matrix = df[['Actuvia', 'Prost', 'Zhuw', 'Actim']]
    w = [4, 2, 6, 0.5]
    Sigma = matrix.cov()
    sigma = Sigma.dot(w)
    sigma = (sigma * w).sum()
    sigma = math.sqrt(sigma)
    quant_P = 1.282 * sigma
    return quant_P

In [103]:
VAR(df)

25.575961187644747

In [107]:
def VAR_by_year(df):
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y', errors='ignore')
    df['year'] = df['date'].dt.year
    arr = []
    for i in [2016, 2017, 2018]:
        v = VAR(df[df['year'] == i])
        arr.append(v)
    return arr

In [135]:
arr = VAR_by_year(df)

In [136]:
arr

[24.526765187941837, 18.75792364166608, 30.633983618474712]

In [115]:
actual = pd.read_csv("actual.csv", header = None, sep = ";")

In [116]:
actual

,0,1
0,2016,29.13
1,2017,27.87
2,2018,25.56


In [137]:
arr_actual = actual[1].values.tolist()

In [138]:
arr_actual

[29.13, 27.87, 25.56]

In [145]:
from sklearn.metrics import mean_squared_error

In [146]:
rms = math.sqrt(mean_squared_error(arr, arr_actual))

In [147]:
print("Root Mean Squared Error: {0}".format(rms))

Root Mean Squared Error: 6.581920086003321


In [149]:
print("Error percentage: {0}".format(rms*100/VAR(df)))

Error percentage: 25.73479071896199
